xlns/examples/arn_generic.py is a hard-coded illustration of training a fully connected MLP with 28*28 input nodes, 100 hidden nodes and 10 output nodes using MNIST digit set. The hidden layer uses RELU and the output layer uses softmax.

The code challenge is to implement a similar size fully connected network (in FP) using the provided features of Pytorch or Tensorflow and compare its convergence with arn_generic.py

In [80]:
# Created using https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html as a template

In [67]:
import numpy
import get_mnist_data
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor

In [68]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor(),
)

In [69]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [70]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
            nn.Softmax(1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
    (3): Softmax(dim=1)
  )
)


In [71]:
loss_fn = nn.NLLLoss() # we use negative log likelihood loss as it is the most effective when working with Softmax()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01 we leave out this optimiser as it is un-generic to compare with arn_generic.py
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0)

In [72]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [73]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [74]:
%%time

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: -0.099548  [   64/60000]
loss: -0.100496  [ 6464/60000]
loss: -0.105713  [12864/60000]
loss: -0.106075  [19264/60000]
loss: -0.107811  [25664/60000]
loss: -0.109539  [32064/60000]
loss: -0.116388  [38464/60000]
loss: -0.109790  [44864/60000]
loss: -0.117082  [51264/60000]
loss: -0.129415  [57664/60000]
Test Error: 
 Accuracy: 41.0%, Avg loss: -0.125925 

Epoch 2
-------------------------------
loss: -0.124308  [   64/60000]
loss: -0.129888  [ 6464/60000]
loss: -0.139337  [12864/60000]
loss: -0.153044  [19264/60000]
loss: -0.154193  [25664/60000]
loss: -0.163596  [32064/60000]
loss: -0.213258  [38464/60000]
loss: -0.183327  [44864/60000]
loss: -0.222405  [51264/60000]
loss: -0.305358  [57664/60000]
Test Error: 
 Accuracy: 47.6%, Avg loss: -0.274966 

Epoch 3
-------------------------------
loss: -0.274817  [   64/60000]
loss: -0.327728  [ 6464/60000]
loss: -0.320134  [12864/60000]
loss: -0.408172  [19264/60000]
loss: -0.345002  [25664/60000]

In [76]:
# We write this here because each time we run the optimiser we get slightly different results
# these are the results that I am going with as they are from using generic SGD rather than Adam as the optimiser
"""
Test Error: 
 Accuracy: 66.0%, Avg loss: -0.619269 

Done!
CPU times: user 2min 25s, sys: 53 ms, total: 2min 25s
Wall time: 36.5 s
"""
print(".")

.


In [77]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = pred[0].argmax(0), y
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "7", Actual: "7"


In [78]:
# Note: This code should not run
# Output from shell:
"""
python arn_generic.py --type float --num_epoch 5
arbitrary base np LNS. Also xl.hstack, xl. routines in softmax
testing new softmax and * instead of @ for delta
works with type float
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
using ./weightin.npz
At Epoch 1:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 1: 0.540000
Val-set accuracy at epoch 1: 0.220000
At Epoch 2:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 2: 0.700000
Val-set accuracy at epoch 2: 0.400000
At Epoch 3:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 3: 0.860000
Val-set accuracy at epoch 3: 0.520000
At Epoch 4:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 4: 0.900000
Val-set accuracy at epoch 4: 0.540000
At Epoch 5:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 5: 0.920000
Val-set accuracy at epoch 5: 0.560000
elasped time=0.14431134099999987
"""
print(".")

.


In [79]:
# Output from shell
"""
python arn_generic.py --type xlns --num_epoch 5
arbitrary base np LNS. Also xl.hstack, xl. routines in softmax
testing new softmax and * instead of @ for delta
works with type xlns
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
using ./weightin.npz
At Epoch 1:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 1: 0.540000
Val-set accuracy at epoch 1: 0.220000
At Epoch 2:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 2: 0.700000
Val-set accuracy at epoch 2: 0.400000
At Epoch 3:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 3: 0.860000
Val-set accuracy at epoch 3: 0.520000
At Epoch 4:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 4: 0.900000
Val-set accuracy at epoch 4: 0.540000
At Epoch 5:
#= 50  xlns b= two   F= 10  B= 1.0006771306930664  batch= 1  lr= 0.01
train-set accuracy at epoch 5: 0.920000
Val-set accuracy at epoch 5: 0.560000
elasped time=539.710699353
"""
print(".")

.


 - arn_generic.py with float: 62% train set accuracy after 5 epochs and 0.14 seconds
 - arn_generic.py with xlns: 56% test set accuracy after 5 epochs and 9 minutes
 - torch with float: 66% test set accuracy after 5 epochs and 2 minutes 25 seconds